In [1]:
# Step 1: Install required packages
!pip install ultralytics
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 23.4 MB/s eta 0:00:00


In [ ]:
import os
from ultralytics import YOLO

# Set up paths
data_yaml = "/content/drive/MyDrive/Licence plate detection model within car detected/Number_plate.v1-roboflow-instant-1--eval-.yolov8/data.yaml"  # You'll need to create this YAML file
output_dir = "/content/drive/MyDrive/Licence plate detection model within car detected/training_results"
os.makedirs(output_dir, exist_ok=True)

# Load a pretrained YOLOv8 model
model = YOLO("yolov8n.pt")  # Using nano version for faster training, can use yolov8s.pt for better accuracy

# Custom training configuration
training_config = {
    "data": data_yaml,
    "epochs": 50,
    "imgsz": 640,
    "batch": 16,
    "optimizer": "auto",
    "lr0": 0.01,  # initial learning rate
    "lrf": 0.01,  # final learning rate (lr0 * lrf)
    "weight_decay": 0.0005,
    "patience": 10,  # reduced patience
    "device": "0",  # use GPU if available
    "project": output_dir,
    "name": "license_plate_detection",
    "exist_ok": True,  # overwrite existing project/name
    "seed": 42,
    "deterministic": True,
    "single_cls": True,  # Set to True since you only have one class ('number_plate')
    "rect": False,
    "cos_lr": False,
    "close_mosaic": 10,
    "resume": False,
    "amp": True,  # automatic mixed precision
    "fraction": 1.0,  # dataset fraction to train on
    "profile": False,
    "freeze": None,  # you can freeze some layers if needed
    "mask_ratio": 4,
    "dropout": 0.0,
    "val": True,  # validate during training
    "save_json": False,
    "save_hybrid": False,
    "conf": 0.25,  # object confidence threshold
    "iou": 0.45,  # Lower IoU threshold might help with small license plates
    "max_det": 300,
    "half": False,
    "dnn": False,
    "plots": True,
    "augment": True,
}

# Train the model
results = model.train(**training_config)

# Validate the model
metrics = model.val()
print(f"mAP50-95: {metrics.box.map}")  # mAP50-95
print(f"mAP50: {metrics.box.map50}")  # mAP50
print(f"mAP75: {metrics.box.map75}")  # mAP75

# Save the trained model
model_path = os.path.join(output_dir, "best_license_plate.pt")
model.export(format="onnx")  # optional: export to ONNX format
model.save(model_path)
print(f"Model saved to {model_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|██████████| 6.25M/6.25M [00:00<00:00, 136MB/s]

WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Licence plate detection model within car detected/Number_plate.v1-roboflow-instant-1--eval-.yolov8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.45, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license_plate_detection, nbs=64, nms=False, opset=None

100%|██████████| 755k/755k [00:00<00:00, 29.7MB/s]

...


40 epochs completed in 0.020 hours.
Optimizer stripped from /content/drive/MyDrive/Licence plate detection model within car detected/training_results/license_plate_detection/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/Licence plate detection model within car detected/training_results/license_plate_detection/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/Licence plate detection model within car detected/training_results/license_plate_detection/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


                   all         21         21      0.996          1      0.995      0.843
Speed: 0.2ms preprocess, 19.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /content/drive/MyDrive/Licence plate detection model within car detected/training_results/license_plate_detection
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 8.4±7.3 MB/s, size: 13.4 KB)


val: Scanning /content/drive/MyDrive/Licence plate detection model within car detected/Number_plate.v1-roboflow-instant-1--eval-.yolov8/valid/labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.80it/s]


                   all         21         21          1          1      0.995      0.841
Speed: 0.2ms preprocess, 36.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /content/drive/MyDrive/Licence plate detection model within car detected/training_results/license_plate_detection
mAP50-95: 0.8412134126984127
mAP50: 0.995
mAP75: 0.995
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/content/drive/MyDrive/Licence plate detection model within car detected/training_results/license_plate_detection/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.56', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 8.3s
WAR

In [6]:
SOURCE_VIDEO_PATH = "/content/drive/MyDrive/two stage detection model/IMG_3575 (1).mov"  # Update your path

In [ ]:
# detects cars and number plates, adds OCR with EasyOCR, gives text as it is
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from tqdm import tqdm
import easyocr
import re

# Paths and Constants
# SOURCE_VIDEO_PATH = "your_input_video.mp4"
TARGET_VIDEO_PATH = "vehicles_with_plates_v3.mp4"
CAR_MODEL_PATH = "yolov8s.pt"
PLATE_MODEL_PATH = "/content/drive/MyDrive/Licence plate detection model within car detected/training_results/best_license_plate.pt"
DETECTION_LINE = [(570, 990), (1770, 990)]

# Load models
car_model = YOLO(CAR_MODEL_PATH)
plate_model = YOLO(PLATE_MODEL_PATH)

# Load OCR
reader = easyocr.Reader(['en'], gpu=True)

# Video IO
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
if not cap.isOpened():
    raise ValueError("Could not open video file")

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter(TARGET_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Helper functions
def line_crossed(prev_y, curr_y, line_y):
    return (prev_y < line_y <= curr_y) or (prev_y > line_y >= curr_y)

def detect_license_plate(vehicle_crop):
    result = plate_model(vehicle_crop, conf=0.4, iou=0.3, verbose=False)
    if not result or result[0].boxes is None:
        return None
    boxes = result[0].boxes.xyxy.cpu().numpy()
    if len(boxes) == 0:
        return None
    x1, y1, x2, y2 = map(int, boxes[0])
    return vehicle_crop[y1:y2, x1:x2]

def clean_text(text):
    # Uppercase, remove non-alphanumeric
    text = text.upper()
    text = re.sub(r'[\s\W_]+', '', text)

    # Convert first character if 'I' to '1'
    if len(text) > 0 and text[0] == 'I':
        text = '1' + text[1:]

    # Convert 3rd or 4th character if '4' to 'A'
    if len(text) >= 4:
        text_list = list(text)
        if text_list[2] == '4':
            text_list[2] = 'A'
        if text_list[3] == '4':
            text_list[3] = 'A'
        text = ''.join(text_list)

    if len(text) == 8:
        text = text[1:]

    return text

# Tracking state
last_positions = defaultdict(lambda: None)
captured_plates = {}

pbar = tqdm(total=total_frames, desc="Processing Video")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    pbar.update(1)
    annotated_frame = frame.copy()

    results = car_model.track(frame, persist=True, classes=[2, 5, 7], conf=0.3, iou=0.5, imgsz=1280, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    ids = results[0].boxes.id.cpu().numpy() if results[0].boxes.id is not None else []

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        tracker_id = int(ids[i])
        curr_bottom_y = y2
        prev_bottom_y = last_positions[tracker_id]

        if prev_bottom_y is not None and line_crossed(prev_bottom_y, curr_bottom_y, DETECTION_LINE[0][1]):
            car_crop = frame[y1:y2, x1:x2].copy()
            plate_crop = detect_license_plate(car_crop)
            if plate_crop is not None:
                resized_plate = cv2.resize(plate_crop, (100, 32))

                # Run OCR and clean result
                result = reader.readtext(resized_plate)
                if result:
                    plate_text = clean_text(result[0][1])
                else:
                    plate_text = "UNKNOWN"

                # Save plate and text
                captured_plates[tracker_id] = (resized_plate, plate_text)

        last_positions[tracker_id] = curr_bottom_y

    # Draw detection line
    cv2.line(annotated_frame, DETECTION_LINE[0], DETECTION_LINE[1], (0, 255, 0), 2)

    # Draw captured plates with text
    top_y = 30
    for idx, (vid, (plate_img, plate_text)) in enumerate(captured_plates.items()):
        id_label = f"ID: {vid}"
        plate_label = plate_text
        x_offset = width - 140
        y_offset = top_y + idx * 80
        if y_offset + 32 >= height:
            break

        # Draw background
        cv2.rectangle(annotated_frame, (x_offset - 5, y_offset - 40), (x_offset + 110, y_offset + 38), (50, 50, 150), -1)

        # Draw ID and plate text
        cv2.putText(annotated_frame, id_label, (x_offset, y_offset - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 1)
        cv2.putText(annotated_frame, plate_label, (x_offset, y_offset + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

        # Show plate image
        annotated_frame[y_offset + 8:y_offset + 8 + 32, x_offset:x_offset + 100] = plate_img

    out.write(annotated_frame)

pbar.close()
cap.release()
out.release()


Processing Video:  99%|█████████▉| 785/793 [01:33<00:00,  8.44it/s]


In [9]:
# detects cars and number plates, adds OCR with EasyOCR, gives text as it is
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from tqdm import tqdm
import easyocr
import re

# Paths and Constants
# SOURCE_VIDEO_PATH = "your_input_video.mp4"
TARGET_VIDEO_PATH = "vehicles_with_plates_v5.mp4"
CAR_MODEL_PATH = "yolov8s.pt"
PLATE_MODEL_PATH = "/content/drive/MyDrive/Licence plate detection model within car detected/training_results/best_license_plate.pt"
DETECTION_LINE = [(570, 990), (1770, 990)]

# Load models
car_model = YOLO(CAR_MODEL_PATH)
plate_model = YOLO(PLATE_MODEL_PATH)

# Load OCR
reader = easyocr.Reader(['en'], gpu=True)

# Video IO
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
if not cap.isOpened():
    raise ValueError("Could not open video file")

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter(TARGET_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Helper functions
def line_crossed(prev_y, curr_y, line_y):
    return (prev_y < line_y <= curr_y) or (prev_y > line_y >= curr_y)

def detect_license_plate(vehicle_crop):
    result = plate_model(vehicle_crop, conf=0.4, iou=0.3, verbose=False)
    if not result or result[0].boxes is None:
        return None, None
    boxes = result[0].boxes.xyxy.cpu().numpy()
    if len(boxes) == 0:
        return None, None
    x1, y1, x2, y2 = map(int, boxes[0])
    return vehicle_crop[y1:y2, x1:x2], (x1, y1, x2, y2)

def clean_text(text):
    # Uppercase, remove non-alphanumeric
    text = text.upper()
    text = re.sub(r'[\s\W_]+', '', text)

    # Convert first character if 'I' to '1'
    if len(text) > 0 and text[0] == 'I':
        text = '1' + text[1:]

    # Convert 3rd or 4th character if '4' to 'A'
    if len(text) >= 4:
        text_list = list(text)
        if text_list[2] == '4':
            text_list[2] = 'A'
        if text_list[3] == '4':
            text_list[3] = 'A'
        text = ''.join(text_list)

    if len(text) == 8:
        text = text[1:]

    return text

# Tracking state
last_positions = defaultdict(lambda: None)
captured_plates = {}

pbar = tqdm(total=total_frames, desc="Processing Video")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    pbar.update(1)
    annotated_frame = frame.copy()

    results = car_model.track(frame, persist=True, classes=[2, 5, 7], conf=0.3, iou=0.5, imgsz=1280, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    ids = results[0].boxes.id.cpu().numpy() if results[0].boxes.id is not None else []

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        tracker_id = int(ids[i])
        curr_bottom_y = y2
        prev_bottom_y = last_positions[tracker_id]

        # Draw car bounding box (blue)
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(annotated_frame, f"ID: {tracker_id}", (x1, y1 - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Always detect and draw license plate (green)
        car_crop = frame[y1:y2, x1:x2].copy()
        plate_crop, plate_coords = detect_license_plate(car_crop)

        if plate_crop is not None:
            # Get plate coordinates in original frame
            plate_x1, plate_y1, plate_x2, plate_y2 = plate_coords
            abs_plate_x1 = x1 + plate_x1
            abs_plate_y1 = y1 + plate_y1
            abs_plate_x2 = x1 + plate_x2
            abs_plate_y2 = y1 + plate_y2

            # Draw license plate bounding box (green)
            cv2.rectangle(annotated_frame, (abs_plate_x1, abs_plate_y1),
                         (abs_plate_x2, abs_plate_y2), (0, 255, 0), 2)

            # Only perform OCR when crossing the line
            if prev_bottom_y is not None and line_crossed(prev_bottom_y, curr_bottom_y, DETECTION_LINE[0][1]):
                resized_plate = cv2.resize(plate_crop, (100, 32))

                # Run OCR and clean result
                result = reader.readtext(resized_plate)
                if result:
                    plate_text = clean_text(result[0][1])
                else:
                    plate_text = "UNKNOWN"

                # Save plate and text
                captured_plates[tracker_id] = (resized_plate, plate_text)

        last_positions[tracker_id] = curr_bottom_y

    # Draw detection line
    cv2.line(annotated_frame, DETECTION_LINE[0], DETECTION_LINE[1], (0, 255, 0), 2)

    # Draw captured plates with text
    top_y = 30
    for idx, (vid, (plate_img, plate_text)) in enumerate(captured_plates.items()):
        id_label = f"ID: {vid}"
        plate_label = plate_text
        x_offset = width - 140
        y_offset = top_y + idx * 80
        if y_offset + 32 >= height:
            break

        # Draw background
        cv2.rectangle(annotated_frame, (x_offset - 5, y_offset - 40),
                     (x_offset + 110, y_offset + 38), (50, 50, 150), -1)

        # Draw ID and plate text
        cv2.putText(annotated_frame, id_label, (x_offset, y_offset - 15),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 1)
        cv2.putText(annotated_frame, plate_label, (x_offset, y_offset + 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

        # Show plate image
        annotated_frame[y_offset + 8:y_offset + 8 + 32, x_offset:x_offset + 100] = plate_img

    out.write(annotated_frame)

pbar.close()
cap.release()
out.release()

Processing Video:  99%|█████████▉| 785/793 [04:12<00:02,  3.11it/s]
